#### Source:

https://docs.djl.ai/docs/demos/aws/sagemaker/large-model-inference/sample-llm/vllm_deploy_llama_13b.html

https://huggingface.co/docs/sagemaker/inference

In [12]:
!pip install -U sagemaker

  Obtaining dependency information for sagemaker from https://files.pythonhosted.org/packages/c9/18/39b659879f4312fc484c79043b0f49fd5446ecd1e1ad90c5043bd6d1c459/sagemaker-2.203.1-py3-none-any.whl.metadata
  Obtaining dependency information for boto3<2.0,>=1.33.3 from https://files.pythonhosted.org/packages/49/ae/1349402cb049379bb9787a1f3302c8ce0af2f588da3ea417575e4a3eac43/boto3-1.34.15-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 867.7 kB/s eta 0:00:00 0:00:01
  Obtaining dependency information for fastapi==0.95.2 from https://files.pythonhosted.org/packages/4e/1a/04887c641b67e6649bde845b9a631f73a7abfbe3afda83957e09b95d88eb/fastapi-0.95.2-py3-none-any.whl.metadata
  Obtaining dependency information for docker from https://files.pythonhosted.org/packages/18/bd/9706c10bb12e05043ef138dc8d412cfd17f29c8df0fb28ad71c96a98785d/docker-7.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic!=1.7,!=1.7.1,!=1.7.2,!=1.7.3,!=1.8,!=1.8.1,

In [2]:
import sagemaker
import boto3
sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20231030T210397')['Role']['Arn']
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name ravi_tej to get Role path.


sagemaker role arn: arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397
sagemaker bucket: sagemaker-ap-south-1-005418323977
sagemaker session region: ap-south-1


In [4]:
!cp -a /Users/ravi.tej/Desktop/ML/Recommendations/arcane/notebooks/llms/prompt-refinement-26-dec-2023/OpenHermes_ApricotTangerineScallop .

In [44]:
%%writefile serving.properties
engine=Python
option.model_id=WintWealth/OpenHermes_ApricotTangerineScallop
option.task=text-generation
option.trust_remote_code=true
option.tensor_parallel_degree=1
option.rolling_batch=vllm
option.dtype=fp16

Writing serving.properties


In [33]:
# %%sh
# mkdir openhermes_apricot
# mv serving.properties openhermes_apricot/

# !mv OpenHermes_ApricotTangerineScallop/* openhermes_apricot/

# %%sh
# tar czvf openhermes_apricot.tar.gz openhermes_apricot/
# rm -rf openhermes_apricot

In [45]:
%%sh
mkdir mymodel
mv serving.properties mymodel/
tar czvf mymodel.tar.gz mymodel/
rm -rf mymodel

a mymodel
a mymodel/serving.properties


In [46]:
from sagemaker import Model, image_uris, serializers, deserializers

In [47]:
image_uri = image_uris.retrieve(
        framework="djl-deepspeed",
        region=sess.boto_session.region_name,
        version="0.25.0"
    )

In [48]:
s3_code_prefix = "article_attribute_llms/OpenHermes_ApricotTangerineScallop"
bucket = sess.default_bucket()  # bucket to house artifacts
code_artifact = sess.upload_data("mymodel.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- &gt; {code_artifact}")

S3 Code or Model tar ball uploaded to --- &gt; s3://sagemaker-ap-south-1-005418323977/article_attribute_llms/OpenHermes_ApricotTangerineScallop/mymodel.tar.gz


In [49]:
model = Model(image_uri=image_uri, model_data=code_artifact, role=role)

In [53]:
instance_type = "ml.g5.2xlarge"
endpoint_name = sagemaker.utils.name_from_base("OpenHermes-ApricotTangerineScallop-vLLM")

In [54]:
endpoint_name

'OpenHermes-ApricotTangerineScallop-vLLM-2024-01-10-15-52-13-940'

In [55]:
model.deploy(initial_instance_count=1,
             instance_type=instance_type,
             endpoint_name=endpoint_name,
             container_startup_health_check_timeout=600
            )

---------------------------*

UnexpectedStatusException: Error hosting endpoint OpenHermes-ApricotTangerineScallop-vLLM-2024-01-10-15-52-13-940: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html

In [ ]:
from datetime import datetime
print(datetime.now())

In [19]:
# our requests and responses will be in json format so we specify the serializer and the deserializer
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=serializers.JSONSerializer(),
)

In [21]:
import time

In [22]:
s = time.time()
predictor.predict(
    {"inputs": "tell me a story of the little red riding hood", "parameters": {"max_tokens":128}}
)
print(f'done in {time.time() - s}')

done in 4.874484062194824
